<a href="https://colab.research.google.com/github/OVP2023/NN/blob/main/%D0%94%D0%97_%D0%9C%D0%BD%D0%BE%D0%B3%D0%BE%D1%81%D0%BB%D0%BE%D0%B9%D0%BD%D0%B0%D1%8F_%D0%BD%D0%B5%D0%B9%D1%80%D0%BE%D0%BD%D0%BD%D0%B0%D1%8F_%D1%81%D0%B5%D1%82%D1%8C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
'''Постройте модель на основе полносвязных слоёв для классификации Fashion MNIST из библиотеки torchvision (это рукописные цифры поэтому будет 10 классов выходов)
Получите качество на тестовой выборке не ниже 88%
Изменяйте архитектуру модели пока качество на тестовой части не будет выше порога. Вариации архитектуры можно реализовать через изменение количества слоёв,
количества нейронов в слоях и использование регуляризации. Можно использовать различные оптимизаторы.'''

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import torch
import numpy as np
import torchvision as tv
import pandas as pd
import time

In [2]:
BATCH_SIZE=256

train_dataset = tv.datasets.FashionMNIST('.', train=True, transform=tv.transforms.ToTensor(), download=True) #train=True - загружается train датасет, transform=tv.transforms.ToTensor() -  картинки преобразуются в тензоры, download=True - они будут скачиваться
test_dataset = tv.datasets.FashionMNIST('.', train=False, transform=tv.transforms.ToTensor(), download=True)
train = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE)
test = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE)

100%|██████████| 26.4M/26.4M [00:03<00:00, 7.91MB/s]
100%|██████████| 29.5k/29.5k [00:00<00:00, 138kB/s]
100%|██████████| 4.42M/4.42M [00:01<00:00, 2.55MB/s]
100%|██████████| 5.15k/5.15k [00:00<00:00, 19.2MB/s]


In [3]:
train_dataset[0][0].shape

torch.Size([1, 28, 28])

In [4]:
test_dataset

Dataset FashionMNIST
    Number of datapoints: 10000
    Root location: .
    Split: Test
    StandardTransform
Transform: ToTensor()

In [17]:
model = torch.nn.Sequential(
    torch.nn.Flatten(),
    torch.nn.Linear(784, 512),
    torch.nn.ReLU(),
    torch.nn.BatchNorm1d(512),
    torch.nn.Linear(512, 256),
    torch.nn.ReLU(),
    torch.nn.BatchNorm1d(256),
    torch.nn.Linear(256, 128),
    torch.nn.ReLU(),
    torch.nn.Dropout(0.5),
    torch.nn.Linear(128, 10)
)

In [18]:
loss = torch.nn.CrossEntropyLoss()
trainer = torch.optim.Adam(model.parameters(), lr=.01)
num_epochs = 10

In [14]:
def train_model():
    for ep in range(num_epochs):
        train_iters, train_passed  = 0, 0
        train_loss, train_acc = 0., 0.
        start=time.time()

        model.train()
        for X, y in train:
            trainer.zero_grad()
            y_pred = model(X)
            l = loss(y_pred, y)
            l.backward()
            trainer.step()
            train_loss += l.item()
            train_acc += (y_pred.argmax(dim=1) == y).sum().item()
            train_iters += 1
            train_passed += len(X)

        test_iters, test_passed  = 0, 0
        test_loss, test_acc = 0., 0.
        model.eval()
        for X, y in test:
            y_pred = model(X)
            l = loss(y_pred, y)
            test_loss += l.item()
            test_acc += (y_pred.argmax(dim=1) == y).sum().item()
            test_iters += 1
            test_passed += len(X)

        print("ep: {}, taked: {:.3f}, train_loss: {}, train_acc: {}, test_loss: {}, test_acc: {}".format(
            ep, time.time() - start, train_loss / train_iters, train_acc / train_passed,
            test_loss / test_iters, test_acc / test_passed)
        )

In [19]:
train_model()

ep: 0, taked: 12.456, train_loss: 0.547701141047985, train_acc: 0.8052333333333334, test_loss: 0.6175060041248799, test_acc: 0.8341
ep: 1, taked: 12.551, train_loss: 0.4209568109918148, train_acc: 0.8505, test_loss: 0.4502679746598005, test_acc: 0.8406
ep: 2, taked: 12.408, train_loss: 0.37890024571976766, train_acc: 0.86525, test_loss: 0.4216412004083395, test_acc: 0.8499
ep: 3, taked: 12.867, train_loss: 0.3614285339700415, train_acc: 0.8712666666666666, test_loss: 0.40694698840379717, test_acc: 0.8675
ep: 4, taked: 12.852, train_loss: 0.3381356673037752, train_acc: 0.87805, test_loss: 0.4372171824797988, test_acc: 0.8687
ep: 5, taked: 12.957, train_loss: 0.3270139271274526, train_acc: 0.88245, test_loss: 0.40796463564038277, test_acc: 0.8659
ep: 6, taked: 13.070, train_loss: 0.3201626374366436, train_acc: 0.8842, test_loss: 0.47260041255503893, test_acc: 0.8637
ep: 7, taked: 13.028, train_loss: 0.30824346390176327, train_acc: 0.8880166666666667, test_loss: 0.4909558787941933, test_a